# Analysis of Citadel Tickets
June 26, 2023  
J. Halverson

## Part 1: Convert the mbox file to CSV

In [ ]:
import sys
import mailbox
import csv
from email.header import decode_header

infile = "Citadel.mbox"
outfile = "Citadel.csv"
writer = csv.writer(open(outfile, "w"))

def get_content(part):
    content = ''
    payload = part.get_payload()
    if isinstance(payload, str):
        content += payload
    else:
        for part in payload:
            content += get_content(part)
    return content

writer.writerow(['date', 'from', 'to', 'subject', 'content'])
for index, message in enumerate(mailbox.mbox(infile)):
    content = get_content(message)
    try:
        subj = decode_header(message['subject'])[0][0]
    except:
        subj = ""

    row = [
        message['date'],
        message['from'],
        message['to'],
        subj,
        content
    ]
    writer.writerow(row)

## Part 2: Analyze the data in pandas

In [ ]:
import pandas as pd
import re

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

In [ ]:
df = pd.read_csv(outfile)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head(2)

In [ ]:
df["from"].value_counts()

The "to" field looks uninteresting:

In [ ]:
df["to"].value_counts().head(20)

In [ ]:
def extract_ticket_id(subject):
    ticket_id = re.findall(r"[0-9][0-9][0-9][0-9][0-9]", subject)
    if ticket_id == []:
        return -1
    else:
        return max(ticket_id)

In [ ]:
df["ticket_id"] = df["subject"].apply(extract_ticket_id)

Let's look a the subject of the tickets that did not have a ticket number -- this is a check that we can ignore these tickets:

In [ ]:
df[df.ticket_id == -1].subject

Let's drop the 59 entries above from the analysis:

In [ ]:
df = df[df.ticket_id != -1]

In [ ]:
df.shape

Now add fields for each person of interest per ticket. Note that some people have multiple values for the "from" people but not any of these four.

In [ ]:
df["Lori"] = df["from"].apply(lambda x: 1 if "lbougher" in x else 0)
df["Colin"] = df["from"].apply(lambda x: 1 if "colinswaney" in x else 0)
df["Eugenia"] = df["from"].apply(lambda x: 1 if "Eugenia" in x else 0)
df["Valerie"] = df["from"].apply(lambda x: 1 if "Valerie Ching" in x else 0)

We can also work by the body and subject of the message:

In [ ]:
df["Infutor"] = df.apply(lambda row: 1 if ("infutor" in row["subject"].lower()) or ("infutor" in row["content"].lower()) else 0, axis=1)
df["L2"] = df.apply(lambda row: 1 if (" L2 " in row["subject"].upper()) or (" L2 " in row["content"].upper()) else 0, axis=1)

In [ ]:
df.head(2)

In [ ]:
d = {"Lori":"sum", "Colin":"sum", "Eugenia":"sum", "Valerie":"sum", "Infutor":"sum", "L2":"sum", "subject":"first"}
gp = df.groupby("ticket_id").agg(d)

The number of unique tickets is:

In [ ]:
gp.shape[0]

Percentage of tickets with that person's involvement:

In [ ]:
print(f'Lori={round(100 * gp[gp["Lori"] > 0].shape[0] / gp.shape[0])}%')

In [ ]:
print(f'Colin={round(100 * gp[gp["Colin"] > 0].shape[0] / gp.shape[0])}%')

In [ ]:
print(f'Eugenia={round(100 * gp[gp["Eugenia"] > 0].shape[0] / gp.shape[0])}%')

In [ ]:
print(f'Valerie={round(100 * gp[gp["Valerie"] > 0].shape[0] / gp.shape[0])}%')

In [ ]:
print(f'Lori|Colin|Eugenia|Valerie={round(100 * gp[(gp["Lori"] > 0) | (gp["Colin"] > 0) | (gp["Eugenia"] > 0) | (gp["Valerie"] > 0)].shape[0] / gp.shape[0])}%')

In [ ]:
print(f'Infutor={round(100 * gp[gp["Infutor"] > 0].shape[0] / gp.shape[0])}%')

In [ ]:
print(f'L2={round(100 * gp[gp["L2"] > 0].shape[0] / gp.shape[0])}%')

The following shows all of th tickets and the number of emails each person sent per ticket:

In [ ]:
gp.reset_index(drop=False, inplace=True)
gp.index += 1
gp